# Hotelling: Optimal carbon price paths

In [30]:
import numpy as np

# Plotly for (interactive) figures
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
# Bugfix for Plotly default export size
pio.kaleido.scope.default_width = None
pio.kaleido.scope.default_height = None
from pyomo.environ import *
from pyomo.dae import *

In [2]:
COLORS_PBL = ['#00AEEF', '#808D1D', '#B6036C', '#FAAD1E', '#3F1464', '#7CCFF2', '#F198C1', '#42B649', '#EE2A23', '#004019', '#F47321', '#511607', '#BA8912', '#78CBBF', '#FFF229', '#0071BB']

### Create abstract model

In [3]:
## Baseline: quadratic baseline. Integral of B(t) gives B_cumulative(t)

def B(t):
    return 1.0 + t / 50. - 0.2 * (t / 50.0)**2

def B_cumulative(t):
    return t + t**2/100.0 - 0.2 * t**3 / (3 * 2500.0)

In [4]:
def MAC(m, t, vanilla=False, **kwargs):
    """ Pyomo implementation of MAC, calling MAC_impl()
    """
    
    a = m.relative_abatement[t]
    aStar = m.aStar[t]
    factor = LOT(t, m.LOT_rate) * (m.LBD_factor[t] if value(m.LBD_rate) < 1 else 1)
    eta = 1 if vanilla else m.eta
    eps = m.eps
    return MAC_impl(a, aStar, factor, eta, eps, vanilla, **kwargs)

def MAC_impl(a, aStar, factor, eta, eps, vanilla=False, gamma=2442, beta=2):
    """ Mathematical formulation of the MAC, in two parts:
    - Vanilla part (power function)
    - Extra inertia/min emissions costs (exponential function)
    """
    
    MAC_vanilla = factor * gamma  * a**beta
    
    if vanilla or eta == 1:
        MAC_inertia = 0
    else:
        MAC_inertia = 0.5 * gamma * eta ** ((a - aStar) / eps)
    
    return MAC_vanilla + MAC_inertia

def AC(m, t, vanilla=False, **kwargs):
    """ Pyomo implementation of abatement costs, calling AC_impl()
    """
    
    a = m.relative_abatement[t]
    aStar = m.aStar[t]
    factor = LOT(t, m.LOT_rate) * (m.LBD_factor[t] if value(m.LBD_rate) < 1 else 1)
    eta = m.eta
    eps = m.eps
    return AC_impl(a, aStar, factor, eta, eps, vanilla, **kwargs)
    
def AC_impl(a, aStar, factor, eta, eps, vanilla=False, gamma=2442, beta=2):
    """ Integral of MAC_impl function, again in two parts (vanilla and extra costs)
    """
    
    AC_vanilla = factor * gamma * a**(beta+1)/(beta+1)
    
    if vanilla or eta == 1:
        AC_inertia = 0
    else:
        AC_inertia = 0.5 * gamma * (eps / log(eta)) * ( eta**((a-aStar)/eps) - eta**(-aStar/eps) )
    return AC_vanilla + AC_inertia


def aStar(E_prev, t, dt, inertia_rate):
    """ Given the emissions at the previous timestep and the inertia rate, 
    returns the maximum relative abatement level aStar"""
    return 1 - (E_prev - dt * inertia_rate) / B(t)

def LOT(t, rate):
    """Learning over time factor"""
    return 1 / (1+rate)**t

In [5]:
def add_constraint(m, constraint):
    """Adds a Pyomo constraint to the model
    
    It first generates a unique name, then adds
    the constraint using this new name
    """
    n = len([_ for _ in m.component_objects()])
    name = 'constraint_{}'.format(n)
    m.add_component(name, constraint)

In [6]:
def create_abstract_model():
    m = AbstractModel()

    constraints = []
    constraints_init = []

    ## Time
    m.tf = Param()
    m.dt = Param()
    m.t = ContinuousSet(bounds=(0, m.tf))
    
    m.simulation = Param(initialize=False)
    m.exog_carbonprice_fct = None


    ## Emissions
    m.emissions = Var(m.t)
    m.relative_abatement = Var(m.t, initialize=0, bounds=(0,4))
    m.cumulative_emissions = Var(m.t)

    m.emissionsdot = DerivativeVar(m.emissions, wrt=m.t)
    m.emissions_prev = Var(m.t, initialize=0)
    
    m.cumulative_emissionsdot = DerivativeVar(m.cumulative_emissions, wrt=m.t)

    constraints.extend([
        lambda m,t: m.emissions[t] == (1-m.relative_abatement[t]) * B(t),
        lambda m,t: m.cumulative_emissionsdot[t] == m.emissions[t] if t > 0 else Constraint.Skip,
        lambda m,t: m.emissions_prev[t] == m.emissions[t] - m.emissionsdot[t] * m.dt if t > 0 else Constraint.Skip
    ])
    
    m.aStar = Var(m.t, initialize=5, bounds=(0, 5))
    m.aStarInertia = Var(m.t, initialize=5)
    m.aStarMinLevel = Var(m.t, initialize=5)
    m.eta = Param()
    m.eps = Param()
    
    ## Inertia
    m.softInertia = Param(initialize=False)
    m.hardInertia = Param(initialize=False)
    m.inertia_val = Param()
    
    constraints.extend([
        lambda m,t: (
            m.aStarInertia[t] == aStar(m.emissions_prev[t], t, m.dt, m.inertia_val)
        ),
        lambda m,t: (
            m.emissionsdot[t] >= -m.inertia_val * B(0)
        ) if value(m.hardInertia) and t > 0 else Constraint.Skip
    ])
    
    ## Minimum emission level
    m.softMinLevel = Param(initialize=False)
    m.hardMinLevel = Param(initialize=False)
    m.min_level = Param()
    
    constraints.extend([
        lambda m,t: m.aStarMinLevel[t] == 1 - m.min_level / B(t),
        lambda m,t: m.emissions[t] >= m.min_level if value(m.hardMinLevel) else Constraint.Skip
    ])
    
    constraints.extend([
        lambda m,t: m.aStar[t] <= m.aStarInertia[t] if value(m.softInertia) else Constraint.Skip,
        lambda m,t: m.aStar[t] <= m.aStarMinLevel[t] if value(m.softMinLevel) and t > 0 else Constraint.Skip,
    ])
    
    ## Only for simulation:
    
    constraints.extend([
        lambda m,t: m.aStar[t] == m.aStarInertia[t] if value(m.softInertia) and value(m.simulation) else Constraint.Skip
    ])
    
    
    
    ## Carbon budget constraint
    m.budget = Param()
    
    constraints_init.extend([
        lambda m: m.emissions[0] == B(0) if not value(m.simulation) else Constraint.Skip,
        lambda m: m.cumulative_emissions[m.tf] <= m.budget if not value(m.simulation) else Constraint.Skip,
        lambda m: m.cumulative_emissions[0] == 0
    ])


    ## Abatement costs
    m.LOT_rate = Param()
    m.LBD_rate = Param()
    m.log_LBD_rate = Param(initialize=log(m.LBD_rate) / log(2))
    m.LBD_factor = Var(m.t)
    m.abatement_costs = Var(m.t)
    m.abatement_costs_vanilla = Var(m.t)
    m.carbonprice = Var(m.t)
    m.carbonprice_vanilla = Var(m.t, initialize=0)

    constraints.extend([
        lambda m,t: (
            m.LBD_factor[t] == ((B_cumulative(t) - m.cumulative_emissions[t]) + 1.0)**m.log_LBD_rate
        ),
        lambda m,t: m.abatement_costs[t] == AC(m, t) * B(t),
        lambda m,t: m.abatement_costs_vanilla[t] == AC(m, t, vanilla=True) * B(t),
        lambda m,t: m.carbonprice[t] == MAC(m, t),
        lambda m,t: m.carbonprice[t] == m.exog_carbonprice_fct(t) if value(m.simulation) else Constraint.Skip,
        lambda m,t: m.carbonprice_vanilla[t] == MAC(m, t, vanilla=True) if not value(m.simulation) else Constraint.Skip
    ])


    ## Optimisation
    m.NPV = Var(m.t)
    m.NPVdot = DerivativeVar(m.NPV, wrt=m.t)
    m.r = Param()
    constraints.append(
        lambda m,t: m.NPVdot[t] == exp(-m.r * t) * m.abatement_costs[t]
    )
    constraints_init.append(lambda m: m.NPV[0] == 0)


    ## Add constraints
    for i, fct in enumerate(constraints):
        add_constraint(m, Constraint(m.t, rule=fct))
    for i, fct in enumerate(constraints_init):
        add_constraint(m, Constraint(rule=fct))


    m.obj = Objective(rule=lambda m: m.NPV[m.tf], sense=minimize)
    
    return m

### Create concrete model

In [7]:
class HotellingModel:
    
    abstract_model = create_abstract_model()
    
    def __init__(self, params, discretize=True):
        tf = params['tf']
        dt = params['dt']
        self.params = params
        self.tf = tf
        self.num_steps = int(np.round(tf/dt))
        self.dt = tf / self.num_steps
        self.m = self.create_instance()
        if discretize:
            self.discretize()
        
    def create_instance(self):
        v = lambda val: {None: val}
        
        instance_data = {None: {
            'tf': v(self.tf),
            'dt': v(self.dt),
            'budget': v(self.params['budget_relative'] * B_cumulative(self.tf)),
            
            'softInertia': v(self.params['softInertia']),
            'hardInertia': v(self.params['hardInertia']),
            'inertia_val': v(self.params['inertia_val']),
            
            'softMinLevel': v(self.params['softMinLevel']),
            'hardMinLevel': v(self.params['hardMinLevel']),
            'min_level': v(self.params['min_level']),
            
            'eps': v(self.params['eps']),
            'eta': v(self.params['eta']),
            'LOT_rate': v(self.params['LOT_rate']),
            'LBD_rate': v(self.params['LBD_rate']),
            'r': v(self.params['r']),
            
            'simulation': v(self.params['simulation'])
        }}
        if self.params['simulation']:
            self.abstract_model.exog_carbonprice_fct = self.params['exog_carbonprice_fct']

        return self.abstract_model.create_instance(instance_data)
        
    def discretize(self):
        discretizer = TransformationFactory('dae.finite_difference')
        discretizer.apply_to(self.m, nfe=self.num_steps, scheme='BACKWARD')
        
    def solve(self, verbose=False):
        opt = SolverFactory('ipopt')
        results = opt.solve(self.m, tee=verbose)
        print('Status: {}, termination: {}'.format(
            results.solver.status, results.solver.termination_condition
        ))
        return self

    def plot(self, fig, row, name, with_dEdt=False, color=None, showlegend=False, with_vanilla=True, opacity=None):
        m = self.m
        
        t_values = np.array(list(m.t))
        t_values_years = 2020 + t_values
        values = lambda var, t_values=t_values: np.array([value(var[t]) for t in t_values])
        specs = lambda hovertext, color_i=0, _showlegend=False: {
            'name': name, 'legendgroup': name if showlegend else None, 'showlegend': _showlegend and showlegend,
            'row': row, 'line_color': COLORS_PBL[color_i] if color == None else color,
            'hovertext': hovertext, 'opacity': opacity
        }
        
        emissions_2020 = 43.3
        
        fig.add_scatter(x=t_values_years, y=values(m.carbonprice), **specs('Carbon price', 1, True), col=1)
        if with_vanilla:
            fig.add_scatter(x=t_values_years, y=values(m.carbonprice_vanilla), **specs('Carbon price vanilla', 1), col=1, line_dash='dot')
        fig.add_scatter(x=t_values_years, y=emissions_2020 * B(t_values), **specs('Baseline', 2), col=2)
        fig.add_scatter(x=t_values_years, y=emissions_2020 * values(m.emissions), **specs('Emissions', 3), col=2)
        fig.add_scatter(x=t_values_years, y=values(m.abatement_costs), **specs('Abatement costs', 4), col=3)
        if with_vanilla:
            fig.add_scatter(x=t_values_years, y=values(m.abatement_costs_vanilla), **specs('Abatement costs vanilla', 4), col=3, line_dash='dot')
        if with_dEdt:
            fig.add_scatter(x=t_values_years[1:], y=values(m.emissionsdot, t_values[1:]), **specs('Emissions dot', 5), col=4)
        return self

In [8]:
LEARNING_LOW = 0.003925
LEARNING_MEDIUM = 0.014823
LEARNING_HIGH = 0.031073

default_params = {
    'tf': 80, 'dt': 2,
    'budget_relative': 0.22, 'r': 0.05,
    'softInertia': False, 'hardInertia': False, 'inertia_val': 0.03,
    'softMinLevel': False, 'hardMinLevel': False, 'min_level': -2,
    'eta': 1, 'eps': 0.0003,
    'LOT_rate': LEARNING_MEDIUM, 'LBD_rate': 1,
    'simulation': False
}

In [9]:
%%time
modelLBD = HotellingModel(dict(default_params,**{'LOT_rate': 0, 'LBD_rate': 0.82})).solve()

Status: ok, termination: optimal
Wall time: 468 ms


In [10]:
%%time
inertia_val = 0.048
modelInertia = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': inertia_val})).solve()
modelInertia_hard = HotellingModel(dict(default_params,**{'hardInertia': True, 'inertia_val': inertia_val})).solve()

# modelInertia_low = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.02, 'inertia_val': inertia_val})).solve()
# modelInertia_high = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.09, 'inertia_val': inertia_val})).solve()
# modelInertia_highest = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.50, 'eps': 0.00004, 'inertia_val': inertia_val})).solve()

Status: ok, termination: optimal
Status: ok, termination: optimal
Wall time: 2.3 s


In [11]:
%%time
min_level = -0.25
modelMinLevel = HotellingModel(dict(default_params,**{'softMinLevel': True, 'eta': 1.035, 'min_level': min_level})).solve()
modelMinLevel_hard = HotellingModel(dict(default_params,**{'hardMinLevel': True, 'min_level': min_level})).solve()

# modelMinLevel_low = HotellingModel(dict(default_params,**{'softMinLevel': True, 'eta': 1.02, 'min_level': min_level})).solve()
# modelMinLevel_high = HotellingModel(dict(default_params,**{'softMinLevel': True, 'eta': 1.09, 'min_level': min_level})).solve()
# modelMinLevel_highest = HotellingModel(dict(default_params,**{'softMinLevel': True, 'eta': 1.50, 'eps': 0.00004, 'min_level': min_level})).solve()

Status: ok, termination: optimal
Status: ok, termination: optimal
Wall time: 1.48 s


In [12]:
%%time
modelCombined = HotellingModel(dict(default_params, **{
    'LOT_rate': 0, 'LBD_rate': 0.82,
    'softInertia': True, 'inertia_val': inertia_val,
    'softMinLevel': True, 'min_level': min_level,
    'eta': 1.035
})).solve()

Status: ok, termination: optimal
Wall time: 668 ms


In [128]:
def add_hotelling(fig, model, row=1, col=1, dt=2, x_i=-5, t0=None, opacity=0.85):
    if t0 is None:
        t0 = 2*dt
    m = model.m
    hotelling = np.array([[t+2020, np.exp(0.05 * (t-t0)) * value(m.carbonprice[t0])] for t in m.t])
    fig.add_scatter(x=hotelling[:,0], y=hotelling[:,1], row=row, col=col, line_color='#BBB', showlegend=False)
    fig.add_annotation(
        x=hotelling[x_i, 0], y=hotelling[x_i, 1], ax=-20, ay=-20,
        opacity=opacity,
        showarrow=True, arrowhead=1, arrowwidth=2, text='Exponential ref. ', xanchor='right',
        row=row, col=col
    )

In [119]:
fig = make_subplots(
    rows=4, cols=3, 
    subplot_titles=[
        '<b>a. Learning by Doing</b>'.ljust(65)+'<br>Carbon price', 'Emissions', 'Abatement costs',
        '<b>b. Inertia</b>'.ljust(77), '', '',
        '<b>c. Minimum emission level</b>'.ljust(58), '', '',
        '<b>d. Combination</b>'.ljust(68)
    ],
    vertical_spacing=0.07, horizontal_spacing=0.075
)

add_hotelling(fig, modelInertia, row=2)
add_hotelling(fig, modelLBD)
add_hotelling(fig, modelCombined, row=4, x_i=-10)
modelLBD.plot(fig, 1, name='LBD')
modelInertia.plot(fig, 2, name='Inertia')
modelMinLevel.plot(fig, 3, name='minEmissionLevel')
modelCombined.plot(fig, 4, name='Combination')


for i, name, dash in [
    (1, 'Carbon price<br>(economic inertia)'.ljust(70), None),                      (2, 'Baseline', None),  (4, 'Abatement costs<br>(economic inertia)', None),
    (1, 'Carbon price<br>(socio-political inertia)', 'dot'), (3, 'Emissions', None), (4, 'Abatement costs<br>(socio-political inertia)', 'dot'),
]:
    fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': COLORS_PBL[i], 'dash': dash}, name=name)

fig.update_yaxes(col=1, matches='y', title='US$/tCO<sub>2</sub>', range=[-301, 4678])
fig.update_yaxes(col=2, matches='y2', title='GtCO<sub>2</sub>/year')
fig.update_yaxes(col=3, matches='y3', title='billion US$/year')
fig.update_yaxes(title_standoff=0)
fig.update_layout(
    margin={'t': 70, 'b': 30, 'l': 30, 'r': 10},
    height=950, width=850,
    legend={'orientation': 'h', 'x': 0.05, 'y': -0.035}
)
fig.show()

In [120]:
fig.write_image('Experiments Hotelling.png', scale=3)
fig.write_image('Experiments Hotelling.svg')

### Sensitivity

In [122]:
modelLBD_low = HotellingModel(dict(default_params,**{'LOT_rate': 0, 'LBD_rate': 0.65, 'dt': 2.5})).solve()
modelLBD_high = HotellingModel(dict(default_params,**{'LOT_rate': 0, 'LBD_rate': 0.95})).solve()

Status: ok, termination: optimal
Status: ok, termination: optimal


In [123]:
modelInertia_low  = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': 0.03})).solve()
modelInertia_high = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': 0.08})).solve()

Status: ok, termination: optimal
Status: ok, termination: optimal


In [124]:
modelInertia_low_lowlearning  = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': 0.03, 'LOT_rate': LEARNING_LOW})).solve()
modelInertia_medium_lowlearning  = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': inertia_val, 'LOT_rate': LEARNING_LOW})).solve()
modelInertia_high_lowlearning = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': 0.08, 'LOT_rate': LEARNING_LOW})).solve()

modelInertia_low_highlearning  = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': 0.03, 'LOT_rate': LEARNING_HIGH})).solve()
modelInertia_medium_highlearning  = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': inertia_val, 'LOT_rate': LEARNING_HIGH})).solve()
modelInertia_high_highlearning = HotellingModel(dict(default_params,**{'softInertia': True, 'eta': 1.035, 'inertia_val': 0.08, 'LOT_rate': LEARNING_HIGH})).solve()

Status: ok, termination: optimal
Status: ok, termination: optimal
Status: ok, termination: optimal
Status: ok, termination: optimal
Status: ok, termination: optimal
Status: ok, termination: optimal


In [125]:
modelMinLevel_low =  HotellingModel(dict(default_params,**{'softMinLevel': True, 'eta': 1.035, 'min_level': 0.0})).solve()
modelMinLevel_high = HotellingModel(dict(default_params,**{'softMinLevel': True, 'eta': 1.035, 'min_level': -0.5})).solve()

Status: ok, termination: optimal
Status: ok, termination: optimal


In [131]:
fig = make_subplots(
    rows=3, cols=3, 
    subplot_titles=[
        '<b>a. Learning by Doing</b>'.ljust(65)+'<br>Carbon price', 'Emissions', 'Abatement costs',
        '<b>b. Inertia</b>'.ljust(77), '', '',
        '<b>c. Minimum emission level</b>'.ljust(58), '', ''
    ],
    vertical_spacing=0.08, horizontal_spacing=0.075
)

add_hotelling(fig, modelLBD, opacity=0.55)
add_hotelling(fig, modelLBD_high, opacity=0.3)
add_hotelling(fig, modelLBD_low, dt=2.5)

modelLBD.plot(fig, 1, name='LBD', with_vanilla=False, opacity=0.67)
modelLBD_high.plot(fig, 1, name='LBD high', with_vanilla=False, opacity=0.33)
modelLBD_low.plot(fig, 1, name='LBD low', with_vanilla=False, opacity=1)

modelInertia.plot(fig, 2, name='Inertia', with_vanilla=True, opacity=.67)
modelInertia_high.plot(fig, 2, name='Inertia high', with_vanilla=True, opacity=.33)
modelInertia_low.plot(fig, 2, name='Inertia low', with_vanilla=True, opacity=1)

# modelInertia_medium_lowlearning.plot(fig, 2, color='mediumvioletred', name='Inertia', with_vanilla=False, opacity=.67)
# modelInertia_high_lowlearning.plot(fig, 2, color='mediumvioletred', name='Inertia high', with_vanilla=False, opacity=.33)
# modelInertia_low_lowlearning.plot(fig, 2, color='mediumvioletred', name='Inertia low', with_vanilla=False, opacity=1)

# modelInertia_medium_highlearning.plot(fig, 2, color='cornflowerblue', name='Inertia', with_vanilla=False, opacity=.67)
# modelInertia_high_highlearning.plot(fig, 2, color='cornflowerblue', name='Inertia high', with_vanilla=False, opacity=.33)
# modelInertia_low_highlearning.plot(fig, 2, color='cornflowerblue', name='Inertia low', with_vanilla=False, opacity=1)

modelMinLevel.plot(fig, 3, name='minEmissionLevel', with_vanilla=True, opacity=.67)
modelMinLevel_high.plot(fig, 3, name='minEmissionLevel high', with_vanilla=True, opacity=.33)
modelMinLevel_low.plot(fig, 3, name='minEmissionLevel low', with_vanilla=True, opacity=1)

for i, name, dash in [
    (1, 'Carbon price<br>(economic inertia)'.ljust(50), None),                      (2, 'Baseline', None),  (4, 'Abatement costs<br>(economic inertia)', None),
    (1, 'Carbon price<br>(socio-political inertia)', 'dot'), (3, 'Emissions', None), (4, 'Abatement costs<br>(socio-political inertia)', 'dot'),
]:
    fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': COLORS_PBL[i], 'dash': dash}, name=name)


fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,1)'}, name='Low end of param range')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,.66)'}, name='Middle of param range')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,.33)'}, name='High end of param range')
fig.add_scatter(x=[None], y=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
    
fig.update_yaxes(col=1, matches='y', title='US$/tCO<sub>2</sub>', range=[-301, 4678])
fig.update_yaxes(col=2, matches='y2', title='GtCO<sub>2</sub>/year')
fig.update_yaxes(col=3, matches='y3', title='billion US$/year')
fig.update_yaxes(title_standoff=0)
fig.update_layout(
    margin={'t': 70, 'b': 30, 'l': 30, 'r': 10},
    height=850, width=850,
    legend={'orientation': 'h', 'x': 0.1, 'y': -0.035}
)
fig.show()

In [132]:
fig.write_image('Experiments Hotelling - sensitivity.png', scale=3)
fig.write_image('Experiments Hotelling - sensitivity.svg')

### Impose exogenous carbon price path

In [23]:
def exogenous_carbonprice(hotellingModel, is_vanilla=True):
    m = hotellingModel.m
    t_values = m.t
    p_values = [value(m.carbonprice_vanilla[t] if is_vanilla else m.carbonprice[t]) for t in t_values]
    return lambda t: np.interp(t, t_values, p_values)

In [24]:
from_model = modelInertia

model_exogenous = HotellingModel(dict(default_params,**{
    'softInertia': True,
    'eta': value(from_model.m.eta), 'eps': value(from_model.m.eps), 'inertia_val': value(from_model.m.inertia_val),
    'simulation': True, 'exog_carbonprice_fct': exogenous_carbonprice(from_model)
})).solve()

Status: ok, termination: optimal


In [25]:
fig = make_subplots(1,3, subplot_titles=('Carbon price', 'Emissions', 'Abatement costs'))
model_exogenous.plot(fig, 1, 'exog1', color='mediumvioletred')
from_model.plot(fig, 1, 'model2', color='cornflowerblue')
fig.show()

# Extra figures

In [29]:
_model = modelInertia.m
t = 50.0

fig = make_subplots(1,1, subplot_titles=('MAC in {:.0f} (x-axis: % reduction)'.format(2020+t),))

E_prev, dt, inertia_rate = value(_model.emissions_prev[t]), value(_model.dt), value(_model.inertia_val)
a_curr = value(_model.relative_abatement[t])
eta, eps = value(_model.eta), value(_model.eps)
learning = LOT(t, value(_model.LOT_rate))
aStarValue = aStar(E_prev, t, dt, inertia_rate)
aPrev = 1 - E_prev/B(t)
gamma = 2442

xmin, xmax = 2*aPrev - aStarValue, 2*aStarValue - aPrev
ymin, ymax = MAC_impl(xmin, 5, learning, 1, 0.001), MAC_impl(xmax, 5, learning, 1, 0.001)+0.75*gamma

shape_style= lambda color='black': {'mode': 'lines', 'line': {'color': color, 'dash': 'dot'}}
fig.add_scatter(x=[aStarValue, aStarValue], y=[ymin, 2*ymax], showlegend=False, **shape_style())
# fig.add_scatter(x=[a_curr, a_curr], y=[ymin, 2*ymax], showlegend=False, **shape_style('gray'))
fig.add_scatter(x=[aPrev, aPrev], y=[ymin, 2*ymax], showlegend=False, **shape_style())

a_values = np.linspace(xmin, xmax, 500)

color1 = lambda i: {'line_color': px.colors.sequential.Plasma[i]}
color2 = lambda i: {'line_color': px.colors.sequential.Viridis[i]}
# fig.add_scatter(x=a_values, y=MAC_impl(a_values, aStarValue, learning, eta=1.02, eps=default_params['eps'], gamma=gamma), name='MAC, Soft 1', **color2(8))
fig.add_scatter(x=a_values, y=MAC_impl(a_values, aStarValue, learning, eta=1.035, eps=default_params['eps'], gamma=gamma), name='MAC (with inertia)', **color2(6))
# fig.add_scatter(x=a_values, y=MAC_impl(a_values, aStarValue, learning, eta=1.09, eps=default_params['eps'], gamma=gamma), name='MAC, Soft 3', **color2(3))
# fig.add_scatter(x=a_values, y=MAC_impl(a_values, aStarValue, learning, eta=1.5, eps=default_params['eps'], gamma=gamma), name='MAC, Soft 4', **color2(0))
fig.add_scatter(x=a_values, y=MAC_impl(a_values, aStarValue, learning, eta=1, eps=0, gamma=gamma), name='MAC (without inertia)', **color1(4))


fig.add_annotation(
    x=aPrev, y=ymin, text="$a(t-\Delta t)$",
    ax=0, ay=50, arrowhead=2, arrowsize=1.5, arrowwidth=1.5
)
fig.add_annotation(
    x=aStarValue, y=ymin, text="$a^*(t)=1-\Big(E(t-\Delta t) - \Delta t \cdot inertia~~rate\Big)/B(t)$",
    ax=0, ay=80, arrowhead=2, arrowsize=1.5, arrowwidth=1.5
)
fig.update_layout(
    xaxis_range=[xmin, xmax],
    yaxis_range=[ymin, ymax], xaxis_tickformat=',.0%',
    yaxis_title='Carbon price (US$)',
    margin={'l': 30, 'r': 30, 'b': 100, 't': 30},
    height=350, width=500
)

In [31]:
fig.write_image('MAC example with soft inertia.png', scale=3)
fig.write_image('MAC example with soft inertia.svg')

### Inertia hotelling test

In [133]:
modelInertia_test1 = HotellingModel(dict(default_params,**{'tf': 80, 'budget_relative': 0.22, 'softInertia': True, 'eta': 1.035, 'inertia_val': inertia_val})).solve()
modelInertia_test2 = HotellingModel(dict(default_params,**{'tf': 50, 'budget_relative': 0.15, 'softInertia': True, 'eta': 1.035, 'inertia_val': inertia_val})).solve()
modelInertia_test3 = HotellingModel(dict(default_params,**{'tf': 100, 'dt': 0.5, 'budget_relative': 0.22, 'softInertia': True, 'eta': 1.055, 'inertia_val': 0.5*inertia_val})).solve()
modelInertia_test4 = HotellingModel(dict(default_params,**{'tf': 100, 'budget_relative': 0.22, 'softInertia': True, 'eta': 1.035, 'inertia_val': 0.5*inertia_val})).solve()

def title(m):
    return 'until {}, dt={}, budget {:.0%}, inertia {}'.format(
        2020+m.params['tf'], m.params['dt'], m.params['budget_relative'], m.params['inertia_val']
    )

fig = make_subplots(rows=4, cols=3, vertical_spacing=0.07, subplot_titles=(
    title(modelInertia_test1), '', '',
    title(modelInertia_test2), '', '',
    title(modelInertia_test3), '', '',
    title(modelInertia_test4), '', ''
))

add_hotelling(fig, modelInertia_test1, row=1)
add_hotelling(fig, modelInertia_test2, row=2, t0=10)
add_hotelling(fig, modelInertia_test3, row=3, t0=5)
add_hotelling(fig, modelInertia_test4, row=4, t0=8)
modelInertia_test1.plot(fig, 1, name='Test 1')
modelInertia_test2.plot(fig, 2, name='Test 2')
modelInertia_test3.plot(fig, 3, name='Test 3')
modelInertia_test4.plot(fig, 4, name='Test 4')

fig.update_layout(
    margin={'t': 30, 'b': 30, 'l': 30, 'r': 10},
    height=1000
)

Status: ok, termination: optimal
Status: ok, termination: optimal
Status: ok, termination: optimal
Status: ok, termination: optimal


In [134]:
fig.write_image('Inertia Hotelling test.png', scale=3)

In [115]:
test = []
m = modelInertia_test4.m
for t0 in np.arange(50, step=2):
    hotelling = np.array([[t+2020, np.exp(0.05 * (t-t0)) * value(m.carbonprice_vanilla[t0]), value(m.carbonprice[t])] for t in m.t])
    test+=[[t0, hotelling[-1,1] / hotelling[-1,2]]]
test = np.array(test)
go.Figure(go.Scatter(x=test[:,0], y=test[:,1]))

In [ ]:
tf=80, budget_relative=0.22, 